## LightGBMRegressor
* This method fit data by using LightGBMRegressor 

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor

import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, log_loss, accuracy_score, roc_curve, roc_auc_score
import gc
import xgboost as xgb
from sklearn.model_selection import train_test_split
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

import time
import sys
import gc
import pickle
sys.version_info

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

## Read data from file, select features.
* data is generated by feature_engineering.ipynb

In [7]:
df = pd.read_pickle("df.pkl")

drop_columns = [c for c in df if c[-1] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'] and c.startswith('target')]
drop_columns += ['ID']
features = df.columns.difference( drop_columns )

f0 = df.date_block_num < 34
f1 = df.date_block_num == 34

train, val = train_test_split(df[f0], test_size=0.2, stratify=df[f0]['target'])
test = df[f1]


## Set up validation strategy

Validation strategy is 34 month for the test set, 33 month for the validation set and 13-32 months for the train.

In [8]:
X_train = train[features]
Y_train = train['target']
X_valid = val[features]
Y_valid = val['target']
X_test = test[features]

del df
gc.collect()


## LightGBM

In [10]:
ts = time.time()

model = LGBMRegressor(
    max_depth = 8,
    n_estimators = 500,
    colsample_bytree=0.7,
    min_child_weight = 300,
    reg_alpha = 0.1,
    reg_lambda = 1,
    random_state = 42,
)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=10, 
    early_stopping_rounds = 40,) # use LGBM's build-in categroical features.

time.time() - ts

Training until validation scores don't improve for 40 rounds.
[10]	training's l2: 0.959247	training's rmse: 0.979412	valid_1's l2: 0.957578	valid_1's rmse: 0.978559
[20]	training's l2: 0.856158	training's rmse: 0.925288	valid_1's l2: 0.854362	valid_1's rmse: 0.924317
[30]	training's l2: 0.825127	training's rmse: 0.908365	valid_1's l2: 0.823605	valid_1's rmse: 0.907527
[40]	training's l2: 0.810359	training's rmse: 0.900199	valid_1's l2: 0.809369	valid_1's rmse: 0.899649
[50]	training's l2: 0.799747	training's rmse: 0.894286	valid_1's l2: 0.799582	valid_1's rmse: 0.894193
[60]	training's l2: 0.792372	training's rmse: 0.890153	valid_1's l2: 0.792902	valid_1's rmse: 0.89045
[70]	training's l2: 0.786367	training's rmse: 0.886773	valid_1's l2: 0.787444	valid_1's rmse: 0.88738
[80]	training's l2: 0.780397	training's rmse: 0.883401	valid_1's l2: 0.781887	valid_1's rmse: 0.884244
[90]	training's l2: 0.775768	training's rmse: 0.880777	valid_1's l2: 0.777792	valid_1's rmse: 0.881925
[100]	trainin

199.57252502441406

## Predict Result and generate output .csv file

In [11]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

X_train_level2 = pd.DataFrame({
    "ID": np.arange(Y_pred.shape[0]), 
    "item_cnt_month": Y_pred
})
X_train_level2.to_csv('./lgb_valid.csv', index=False)

submission = pd.DataFrame({
    "ID": np.arange(Y_test.shape[0]), 
    "item_cnt_month": Y_test
})
submission.to_csv('./lgb_submission.csv', index=False)

### Public LeaderBoard score: 0.90501